In [1]:
# importing basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
from tqdm import tqdm

import tensorflow as tf

2024-09-03 10:36:01.298887: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-03 10:36:01.299026: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-03 10:36:01.433796: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Load the data

In [3]:
# loading the train data
train_path = '/kaggle/input/mn-traffic-1l-3t-processed/traffic-np-3t-l1-processed/train/'
filename = 'X_train_3timesteps_lag1only.npy'
X_train = np.load(train_path + filename)

filename = 'y_train_3timesteps_lag1only.npy'
y_train = np.load(train_path + filename)

X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)

X_train_tensor.shape, y_train_tensor.shape

(TensorShape([6074003, 3, 69]), TensorShape([6074003]))

In [4]:
X_train_tensor[0], y_train_tensor[0]

(<tf.Tensor: shape=(3, 69), dtype=float32, numpy=
 array([[-0.8088384 , -0.36274067, -0.33453488, -0.18419866,  1.2734506 ,
          0.69969475,  0.29899016, -2.0042899 , -1.2237915 , -1.5611751 ,
         -1.1824385 ,  1.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
          1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.25881904,  0.9659258 ,  0.43388373, -0.90096885,  

In [5]:
# loading the valid data
valid_path = '/kaggle/input/mn-traffic-1l-3t-processed/traffic-np-3t-l1-processed/valid/'
filename = 'X_valid_3timesteps_lag1only.npy'
X_valid = np.load(valid_path + filename)

filename = 'y_valid_3timesteps_lag1only.npy'
y_valid = np.load(valid_path + filename)

X_valid_tensor = tf.convert_to_tensor(X_valid, dtype=tf.float32)
y_valid_tensor = tf.convert_to_tensor(y_valid, dtype=tf.float32)

X_valid_tensor.shape, y_train_tensor.shape

(TensorShape([1003233, 3, 69]), TensorShape([6074003]))

In [6]:
X_valid_tensor[0], y_valid_tensor[0]

(<tf.Tensor: shape=(3, 69), dtype=float32, numpy=
 array([[-0.6261748 , -0.6461904 , -0.38455313, -0.27443257,  0.3584298 ,
         -0.38962212,  0.32877734, -0.95747787, -0.01517243, -1.6708986 ,
         -1.2276366 ,  1.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
          1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  1.        , -0.7818315 ,  0.6234898 , -

In [7]:
# loading the test data
test_path = '/kaggle/input/mn-traffic-1l-3t-processed/traffic-np-3t-l1-processed/test/'
filename = 'X_test_3timesteps_lag1only.npy'
X_test = np.load(test_path + filename)

filename = 'y_test_3timesteps_lag1only.npy'
y_test = np.load(test_path + filename)

X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)

X_test_tensor.shape, y_test_tensor.shape

(TensorShape([1019634, 3, 69]), TensorShape([1019634]))

In [8]:
X_test_tensor[0], y_test_tensor[0]

(<tf.Tensor: shape=(3, 69), dtype=float32, numpy=
 array([[ 1.1150675 ,  1.1210097 , -0.3197022 , -0.17390785, -0.12041119,
          1.0266287 , -1.1929966 , -0.39753765,  0.        , -1.675692  ,
         -1.2540781 ,  1.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
          1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  1.        , -0.9749279 , -0.22252093,  

# Model Building

In [9]:
# importing libraries for machine learning models
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score, mean_absolute_error

### Preparing the data

### Helper Function for prediction

In [10]:
# # Create one for deep learning
# def predict_ml(pipeline_model, X):
#     pred = []
#     X['volume_lag1'] = 0
#     X['volume_lag2'] = 0
#     X['volume_lag3'] = 0
    
#     for i in range(len(X)):
#         predicition = pipeline_model.predict(X.iloc[[i]])[0]
        
#         X.loc[i+1, 'volume_lag1'] = predicition
#         pred.append(predicition)
        
#         if i > 0:
#             X.loc[i+1, 'volume_lag2'] = X.loc[i, 'volume_lag1']
        
#         if i > 1:
#             X.loc[i+1, 'volume_lag3'] = X.loc[i, 'volume_lag2']
        
    
#     return pred

In [11]:
def mape(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
    
    epsilon = tf.keras.backend.epsilon()  
    abs_error = tf.abs(y_true - y_pred)
    percentage_error = abs_error / (tf.abs(y_true) + epsilon)
    
    mask = tf.cast(y_true != 0, dtype=tf.float32)
    masked_percentage_error = percentage_error * mask
    
    mape_value = tf.reduce_sum(masked_percentage_error) / tf.reduce_sum(mask)
    
    return mape_value

In [12]:
results = {}

results['train'] = {}
results['valid'] = {}

results

{'train': {}, 'valid': {}}

### Helper Function for Deep Learning Models

In [13]:
def rnn_model(X, units:int=64, add_rnn:bool=False, num_rnn:int=1, divide:int=2, dropout=0.0, num_dense=None):
    r = False
    if add_rnn:
        r = True
    
    model = Sequential()
    model.add(SimpleRNN(units, return_sequences=r, input_shape=X.shape[1:]))
    model.add(LayerNormalization(axis=-1))
    model.add(Dropout(dropout))
    
    u = units // divide
    for i in range(num_rnn):
        if ((u // divide == 1) and (divide > 1)) or (i==num_rnn-1):
            r = False  
        if add_rnn:
            model.add(SimpleRNN(u, return_sequences=r))
            model.add(LayerNormalization(axis=-1))
            model.add(Dropout(dropout))
        if u==1:
            print('Number of RNNs given is not possible except if you change the divide parameter!')
            break
        
        u//=divide
    
    i=u//2
    d=1
    while (i>1) and ((num_dense is None) or (d<=num_dense)):
        model.add(Dense(i, activation=LeakyReLU(alpha=0.01)))
        model.add(Dropout(dropout))
        i=i//2
        d+=1
        
    
    model.add(Dense(1, activation='relu'))
    return model

In [14]:
def lstm_model(X, units:int=64, add_lstm:bool=False, num_lstm:int=1, divide:int=2, dropout=0.0):
    r = False
    if add_lstm:
        r = True
    
    model = Sequential()
    model.add(LSTM(units, return_sequences=r, input_shape=X.shape[1:]))
    model.add(LayerNormalization(axis=-1))
    model.add(Dropout(dropout))
    
    u = units // divide
    for i in range(num_lstm):
        if ((u // divide == 1) and (divide > 1)) or (i==num_lstm-1):
            r = False  
        if add_lstm:
            model.add(LSTM(u, return_sequences=r))
            model.add(LayerNormalization(axis=-1))
            model.add(Dropout(dropout))
        if u==1:
            print('Number of LSTMs given is not possible except if you change the divide parameter!')
            break
        
        u//=divide
    
    i=u//2
    while i>1:
        model.add(Dense(i, activation=LeakyReLU(alpha=0.01)))
        model.add(Dropout(dropout))
        i=i//2
    
    model.add(Dense(1, activation='relu'))
    return model

In [15]:
def gru_model(X, units:int=64, add_gru:bool=False, num_gru:int=1, divide:int=2, dropout=0.0):
    r = False
    if add_gru:
        r = True
    
    model = Sequential()
    model.add(GRU(units, return_sequences=r, input_shape=X.shape[1:]))
    model.add(LayerNormalization(axis=-1))
    model.add(Dropout(dropout))
    
    u = units // divide
    for i in range(num_gru):
        if ((u // divide == 1) and (divide > 1)) or (i==num_gru-1):
            r = False  
        if add_gru:
            model.add(GRU(u, return_sequences=r))
            model.add(LayerNormalization(axis=-1))
            model.add(Dropout(dropout))
        if u==1:
            print('Number of GRUs given is not possible except if you change the divide parameter!')
            break
        
        u//=divide
    
    i=u//2
    while i>1:
        model.add(Dense(i, activation=LeakyReLU(alpha=0.01)))
        model.add(Dropout(dropout))
        i=i//2
    
    model.add(Dense(1, activation='relu'))
    return model

In [16]:
def convlstm_model(X, filters:int=64, kernel_size:int=3, add_convlstm:bool=False, num_convlstm:int=1, divide:int=2, dropout=0.0):
    r = False
    if add_convlstm:
        r = True
    
    model = Sequential()
    model.add(ConvLSTM1D(filters=filters, kernel_size=kernel_size, padding='same', return_sequences=r, input_shape=X.shape[1:]))
    model.add(LayerNormalization(axis=-1))
    model.add(Dropout(dropout))
    
    f = filters // divide
    for i in range(num_convlstm):
        if ((f // divide == 1) and (divide > 1)) or (i==num_convlstm-1):
            r = False  
        if add_convlstm:
            model.add(ConvLSTM1D(filters=f, kernel_size=kernel_size, padding='same', return_sequences=r))
            model.add(LayerNormalization(axis=-1))
            model.add(Dropout(dropout))
        if f==1:
            print('Number of ConvLSTMs given is not possible except if you change the divide parameter!')
            break
        
        f//=divide
    
    model.add(Flatten())
    
    i=f//2
    while i>1:
        model.add(Dense(i, activation=LeakyReLU(alpha=0.01)))
        model.add(Dropout(dropout))
        i=i//2
    
    model.add(Dense(1, activation='relu'))
    return model

In [17]:
from tensorflow.keras.callbacks import Callback
class LearningRateLogger(Callback):
    def on_epoch_end(self, epoch, logs=None):
        current_lr = self.model.optimizer.learning_rate 
        print(f" - Learning Rate at end of epoch {epoch + 1}: {current_lr}")

## Deep Learning Models

In [18]:
# importing the deep learning libraries
from tensorflow.keras.layers import LeakyReLU, Flatten, Dropout, LayerNormalization, Dense, SimpleRNN, LSTM, GRU, ConvLSTM1D
from tensorflow.keras.models import Sequential

from tensorflow.keras.losses import MSE, MAPE, MAE
from tensorflow.keras.metrics import MeanAbsolutePercentageError, RootMeanSquaredError, R2Score, MeanAbsoluteError
from tensorflow.keras.optimizers import Adam, AdamW, Adamax

from tensorflow.keras.optimizers.schedules import CosineDecay

RNN

In [19]:
# Define the model
num_rnn = 0
rnn = rnn_model(X_train_tensor, units=24, add_rnn=False, num_rnn=num_rnn, divide=1, num_dense=2)
rnn.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/conda/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 24)             │         2,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (None, 24)             │            48 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 12)             │           300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 12)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │            78 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,689 (10.50 KB)

 Trainable params: 2,689 (10.50 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Define the parameters
batch_size = 256
epochs = 15
lr = 1e-3

In [21]:
steps_per_epoch = int(np.ceil(X_train_tensor.shape[0] / batch_size))

initial_learning_rate = lr * 0.1
warmup_target = lr
warmup_steps = steps_per_epoch 
decay_steps = steps_per_epoch * epochs

lr_schedule = CosineDecay(initial_learning_rate=initial_learning_rate, 
                          decay_steps=decay_steps, 
                          warmup_target=warmup_target, 
                          warmup_steps=warmup_steps)

In [22]:
optimizer = Adam(learning_rate=lr_schedule)

In [23]:
# Training the model
rnn.compile(optimizer=optimizer, loss='mse', metrics=[RootMeanSquaredError(name='rmse'), R2Score(name='r2_score'), MeanAbsoluteError(name='mae')])
history = rnn.fit(X_train_tensor, y_train_tensor, batch_size=batch_size, epochs=epochs, validation_data=(X_valid_tensor, y_valid_tensor), callbacks=[LearningRateLogger()])

Epoch 1/15
   67/23727 ━━━━━━━━━━━━━━━━━━━━ 55s 2ms/step - loss: 522623.5625 - mae: 550.2830 - r2_score: -1.3755 - rmse: 722.8985

I0000 00:00:1725359835.935352     102 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


23727/23727 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 144736.6094 - mae: 199.1908 - r2_score: 0.3425 - rmse: 352.1826 - Learning Rate at end of epoch 1: 0.0010000000474974513
23727/23727 ━━━━━━━━━━━━━━━━━━━━ 70s 3ms/step - loss: 144732.5000 - mae: 199.1865 - r2_score: 0.3425 - rmse: 352.1769 - val_loss: 10466.3828 - val_mae: 71.1947 - val_r2_score: 0.9563 - val_rmse: 102.3053
Epoch 2/15
23715/23727 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6732.8804 - mae: 51.0825 - r2_score: 0.9694 - rmse: 82.0480 - Learning Rate at end of epoch 2: 0.0009890738874673843
23727/23727 ━━━━━━━━━━━━━━━━━━━━ 62s 3ms/step - loss: 6732.7573 - mae: 51.0821 - r2_score: 0.9694 - rmse: 82.0472 - val_loss: 8716.3047 - val_mae: 63.8124 - val_r2_score: 0.9636 - val_rmse: 93.3612
Epoch 3/15
23727/23727 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6120.6011 - mae: 48.8692 - r2_score: 0.9722 - rmse: 78.2336 - Learning Rate at end of epoch 3: 0.0009567727101966739
23727/23727 ━━━━━━━━━━━━━━━━━━━━ 61s 3ms/step - loss: 6120.598

In [ ]:
# predicting the values
y_train_pred = rnn.predict(X_train_tensor)
y_valid_pred = rnn.predict(X_valid_tensor)

 21879/189813 ━━━━━━━━━━━━━━━━━━━━ 3:40 1ms/step

In [ ]:
y_train_pred = pd.DataFrame(y_train_pred)
y_train_pred.head()

In [ ]:
y_valid_pred = pd.DataFrame(y_valid_pred)
y_valid_pred.head()

In [ ]:
# Calculating the metrics
train_mse = mean_squared_error(pd.DataFrame(y_train), y_train_pred)
train_mae = mean_absolute_error(pd.DataFrame(y_train), y_train_pred)
train_mape = mape(pd.DataFrame(y_train), y_train_pred)
train_r2 = r2_score(pd.DataFrame(y_train), y_train_pred)

valid_mse = mean_squared_error(pd.DataFrame(y_valid), y_valid_pred)
valid_mae = mean_absolute_error(pd.DataFrame(y_valid), y_valid_pred)
valid_mape = mape(pd.DataFrame(y_valid), y_valid_pred)
valid_r2 = r2_score(pd.DataFrame(y_valid), y_valid_pred)

print(f'Train MSE: {train_mse}, Train MAE: {train_mae}, Train MAPE: {train_mape}, Train R2: {train_r2}')
print(f'Validation MSE: {valid_mse},Validation MAE: {valid_mae}, Validation MAPE: {valid_mape}, Validation R2: {valid_r2}')

In [ ]:
results['train']['mse'] = [train_mse]
results['train']['mape'] = [train_mape]
results['train']['r2'] = [train_r2]

results['valid']['mse'] = [valid_mse]
results['valid']['mape'] = [valid_mape]
results['valid']['r2'] = [valid_r2]

results

In [ ]:
# Plot the training loss
plt.plot(history.history['loss'])
plt.show()

In [ ]:
# Plot the validation loss
plt.plot(history.history['val_loss'])
plt.show()

In [ ]:
# Plot the training rmse loss
plt.plot(history.history['rmse'])
plt.show()

In [ ]:
# Plot the validation rmse loss
plt.plot(history.history['val_rmse'])
plt.show()

In [ ]:
# Plot the training R2 Score
plt.plot(history.history['r2_score'])
plt.show()

In [ ]:
# Plot the validation R2 Score
plt.plot(history.history['val_r2_score'])
plt.show()

In [ ]:
# Plotting the predictions
plt.figure(figsize=(12, 6))
plt.plot(y_valid, label='Actual')
plt.plot(y_valid_pred, label='Predicted')
plt.legend()
plt.show()

In [ ]:
# Plotting for one station, and direction
mask_station = X_valid[:,0,11]==1
mask_dir = X_valid[:,0,52]==1
mask_lane = X_valid[:,0,55]==1
mask = mask_station & mask_dir & mask_lane

plt.figure(figsize=(12, 6))
plt.plot(y_valid[mask], label='Actual')
plt.plot(y_valid_pred[mask], label='Predicted')
plt.legend()
plt.show()

In [ ]:
# Save the model
rnn.save(f'rnn_{num_rnn+1}.h5')

# Save the weights
rnn.save_weights('ckpt.weights.h5')

In [ ]:
import pickle
with open(f'/kaggle/working/rnn_{num_rnn+1}', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
with open(f'/kaggle/working/rnn_{num_rnn+1}', "rb") as file_pi:
    h = pickle.load(file_pi)

### LSTM Classic:

In [ ]:
# # Define the model
# num_lstm = 0
# lstm = lstm_model(X_train_tensor, units=16, add_lstm=False, num_lstm=num_lstm, divide=1)
# lstm.summary()

In [ ]:
# # Define the parameters
# batch_size = 64
# epochs = 15
# lr = 1e-3

In [ ]:
# steps_per_epoch = int(np.ceil(X_train_tensor.shape[0] / batch_size))

# initial_learning_rate = lr * 0.1
# warmup_target = lr
# warmup_steps = steps_per_epoch * 2
# decay_steps = steps_per_epoch * epochs

# lr_schedule = CosineDecay(initial_learning_rate=initial_learning_rate, 
#                           decay_steps=decay_steps, 
#                           warmup_target=warmup_target, 
#                           warmup_steps=warmup_steps)

In [ ]:
# optimizer = Adam(learning_rate=lr_schedule)

In [ ]:
# # Training the model
# lstm.compile(optimizer=optimizer, loss='mse', metrics=[RootMeanSquaredError(name='rmse'), R2Score(name='r2_score'), MeanAbsoluteError(name='mae')])
# history = lstm.fit(X_train_tensor, y_train_tensor, batch_size=batch_size, epochs=epochs, validation_data=(X_valid_tensor, y_valid_tensor), callbacks=[LearningRateLogger()])

In [ ]:
# # predicting the values
# y_train_pred = lstm.predict(X_train_tensor)
# y_valid_pred = lstm.predict(X_valid_tensor)

In [ ]:
# y_train_pred = pd.DataFrame(y_train_pred)
# y_train_pred.head()

In [ ]:
# y_valid_pred = pd.DataFrame(y_valid_pred)
# y_valid_pred.head()

In [ ]:
# # Calculating the metrics
# train_mse = mean_squared_error(pd.DataFrame(y_train), y_train_pred)
# train_mae = mean_absolute_error(pd.DataFrame(y_train), y_train_pred)
# train_mape = mape(pd.DataFrame(y_train), y_train_pred)
# train_r2 = r2_score(pd.DataFrame(y_train), y_train_pred)

# valid_mse = mean_squared_error(pd.DataFrame(y_valid), y_valid_pred)
# valid_mae = mean_absolute_error(pd.DataFrame(y_valid), y_valid_pred)
# valid_mape = mape(pd.DataFrame(y_valid), y_valid_pred)
# valid_r2 = r2_score(pd.DataFrame(y_valid), y_valid_pred)

# print(f'Train MSE: {train_mse}, Train MAE: {train_mae}, Train MAPE: {train_mape}, Train R2: {train_r2}')
# print(f'Validation MSE: {valid_mse},Validation MAE: {valid_mae}, Validation MAPE: {valid_mape}, Validation R2: {valid_r2}')

In [ ]:
# results['train']['mse'] = [train_mse]
# results['train']['mape'] = [train_mape]
# results['train']['r2'] = [train_r2]

# results['valid']['mse'] = [valid_mse]
# results['valid']['mape'] = [valid_mape]
# results['valid']['r2'] = [valid_r2]

# results

In [ ]:
# # Plot the training loss
# plt.plot(history.history['loss'])
# plt.show()

In [ ]:
# # Plot the validation loss
# plt.plot(history.history['val_loss'])
# plt.show()

In [ ]:
# # Plot the training rmse loss
# plt.plot(history.history['rmse'])
# plt.show()

In [ ]:
# # Plot the validation rmse loss
# plt.plot(history.history['val_rmse'])
# plt.show()

In [ ]:
# # Plot the training R2 Score
# plt.plot(history.history['r2_score'])
# plt.show()

In [ ]:
# # Plot the validation R2 Score
# plt.plot(history.history['val_r2_score'])
# plt.show()

In [ ]:
# # Plotting the predictions
# plt.figure(figsize=(12, 6))
# plt.plot(y_valid, label='Actual')
# plt.plot(y_valid_pred, label='Predicted')
# plt.legend()
# plt.show()

In [ ]:
# # Plotting for one station, and direction
# mask_station = X_valid[:,0,11]==1
# mask_dir = X_valid[:,0,52]==1
# mask_lane = X_valid[:,0,55]==1
# mask = mask_station & mask_dir & mask_lane

# plt.figure(figsize=(12, 6))
# plt.plot(y_valid[mask], label='Actual')
# plt.plot(y_valid_pred[mask], label='Predicted')
# plt.legend()
# plt.show()

In [ ]:
# # Save the model
# lstm.save(f'lstm_{num_lstm+1}.h5')

# # Save the weights
# lstm.save_weights('ckpt.weights.h5')

In [ ]:
# import pickle
# with open(f'/kaggle/working/lstm_{num_lstm+1}', 'wb') as file_pi:
#     pickle.dump(history.history, file_pi)

In [ ]:
# with open(f'/kaggle/working/lstm_{num_lstm+1}', "rb") as file_pi:
#     h = pickle.load(file_pi)

In [ ]:
# h

### GRU:

In [ ]:
# # Define the model
# num_gru = 0
# gru = gru_model(X_train_tensor, units=64, add_gru=False, num_gru=num_gru, divide=1, dropout=0.2)
# gru.summary()

In [ ]:
# # Define the parameters
# batch_size = 64
# epochs = 15
# lr = 1e-3

In [ ]:
# steps_per_epoch = int(np.ceil(X_train_tensor.shape[0] / batch_size))

# initial_learning_rate = lr * 0.1
# warmup_target = lr
# warmup_steps = steps_per_epoch * 2
# decay_steps = steps_per_epoch * epochs

# lr_schedule = CosineDecay(initial_learning_rate=initial_learning_rate, 
#                           decay_steps=decay_steps, 
#                           warmup_target=warmup_target, 
#                           warmup_steps=warmup_steps)

In [ ]:
# optimizer = Adam(learning_rate=lr_schedule)

In [ ]:
# # Training the model
# gru.compile(optimizer=optimizer, loss='mae', metrics=[RootMeanSquaredError(name='rmse'), R2Score(name='r2_score')])
# history = gru.fit(X_train_tensor, y_train_tensor, batch_size=batch_size, epochs=epochs, validation_data=(X_valid_tensor, y_valid_tensor), callbacks=[LearningRateLogger()])

In [ ]:
# # predicting the values
# y_train_pred = gru.predict(X_train_tensor)
# y_valid_pred = gru.predict(X_valid_tensor)

In [ ]:
# y_train_pred = pd.DataFrame(y_train_pred)
# y_train_pred.head()

In [ ]:
# y_valid_pred = pd.DataFrame(y_valid_pred)
# y_valid_pred.head()

In [ ]:
# # Calculating the metrics
# train_mse = mean_squared_error(pd.DataFrame(y_train), y_train_pred)
# train_mape = mape(pd.DataFrame(y_train), y_train_pred)
# train_r2 = r2_score(pd.DataFrame(y_train), y_train_pred)

# valid_mse = mean_squared_error(pd.DataFrame(y_valid), y_valid_pred)
# valid_mape = mape(pd.DataFrame(y_valid), y_valid_pred)
# valid_r2 = r2_score(pd.DataFrame(y_valid), y_valid_pred)

# print(f'Train MSE: {train_mse}, Train MAPE: {train_mape}, Train R2: {train_r2}')
# print(f'Validation MSE: {valid_mse}, Validation MAPE: {valid_mape}, Validation R2: {valid_r2}')

In [ ]:
# results['train']['mse'] = [train_mse]
# results['train']['mape'] = [train_mape]
# results['train']['r2'] = [train_r2]

# results['valid']['mse'] = [valid_mse]
# results['valid']['mape'] = [valid_mape]
# results['valid']['r2'] = [valid_r2]

# results

In [ ]:
# # Plot the training loss
# plt.plot(history.history['loss'])
# plt.show()

In [ ]:
# # Plot the validation loss
# plt.plot(history.history['val_loss'])
# plt.show()

In [ ]:
# # Plot the training rmse loss
# plt.plot(history.history['rmse'])
# plt.show()

In [ ]:
# # Plot the validation rmse loss
# plt.plot(history.history['val_rmse'])
# plt.show()

In [ ]:
# # Plot the training R2 Score
# plt.plot(history.history['r2_score'])
# plt.show()

In [ ]:
# # Plot the validation R2 Score
# plt.plot(history.history['val_r2_score'])
# plt.show()

In [ ]:
# # Plotting the predictions
# plt.figure(figsize=(12, 6))
# plt.plot(y_valid, label='Actual')
# plt.plot(y_valid_pred, label='Predicted')
# plt.legend()
# plt.show()

In [ ]:
# # Plotting for one station, and direction
# mask_station = X_valid[:,0,11]==1
# mask_dir = X_valid[:,0,52]==1
# mask_lane = X_valid[:,0,55]==1
# mask = mask_station & mask_dir & mask_lane

# plt.figure(figsize=(12, 6))
# plt.plot(y_valid[mask], label='Actual')
# plt.plot(y_valid_pred[mask], label='Predicted')
# plt.legend()
# plt.show()

In [ ]:
# # Save the model
# gru.save(f'gru_{num_gru+1}.h5')

# # Save the weights
# gru.save_weights('ckpt.weights.h5')

In [ ]:
# import pickle
# with open(f'/kaggle/working/gru_{num_gru+1}', 'wb') as file_pi:
#     pickle.dump(history.history, file_pi)

In [ ]:
# with open(f'/kaggle/working/gru_{num_gru+1}', "rb") as file_pi:
#     h = pickle.load(file_pi)

In [ ]:
# h

### ConvLSTM:

In [ ]:
# X_train_tensor = tf.expand_dims(X_train_tensor, axis=-1)
# X_valid_tensor = tf.expand_dims(X_valid_tensor, axis=-1)
# X_test_tensor = tf.expand_dims(X_test_tensor, axis=-1)

In [ ]:
# # Define the model
# num_convlstm = 0
# convlstm = convlstm_model(X_train_tensor, filters=128, kernel_size=3, add_convlstm=False, num_convlstm=num_convlstm, divide=1, dropout=0.2)
# convlstm.summary()

In [ ]:
# # Define the parameters
# batch_size = 64
# epochs = 15
# lr = 1e-3

In [ ]:
# steps_per_epoch = int(np.ceil(X_train_tensor.shape[0] / batch_size))

# initial_learning_rate = lr * 0.1
# warmup_target = lr
# warmup_steps = steps_per_epoch * 2
# decay_steps = steps_per_epoch * epochs

# lr_schedule = CosineDecay(initial_learning_rate=initial_learning_rate, 
#                           decay_steps=decay_steps, 
#                           warmup_target=warmup_target, 
#                           warmup_steps=warmup_steps)

In [ ]:
# optimizer = Adam(learning_rate=lr_schedule)

In [ ]:
# # Training the model
# convlstm.compile(optimizer=optimizer, loss='mae', metrics=[RootMeanSquaredError(name='rmse'), R2Score(name='r2_score')])
# history = convlstm.fit(X_train_tensor, y_train_tensor, batch_size=batch_size, epochs=epochs, validation_data=(X_valid_tensor, y_valid_tensor), callbacks=[LearningRateLogger()])

In [ ]:
# # predicting the values
# y_train_pred = convlstm.predict(X_train_tensor)
# y_valid_pred = convlstm.predict(X_valid_tensor)

In [ ]:
# y_train_pred = pd.DataFrame(y_train_pred)
# y_train_pred.head()

In [ ]:
# y_valid_pred = pd.DataFrame(y_valid_pred)
# y_valid_pred.head()

In [ ]:
# # Calculating the metrics
# train_mse = mean_squared_error(pd.DataFrame(y_train), y_train_pred)
# train_mape = mape(pd.DataFrame(y_train), y_train_pred)
# train_r2 = r2_score(pd.DataFrame(y_train), y_train_pred)

# valid_mse = mean_squared_error(pd.DataFrame(y_valid), y_valid_pred)
# valid_mape = mape(pd.DataFrame(y_valid), y_valid_pred)
# valid_r2 = r2_score(pd.DataFrame(y_valid), y_valid_pred)

# print(f'Train MSE: {train_mse}, Train MAPE: {train_mape}, Train R2: {train_r2}')
# print(f'Validation MSE: {valid_mse}, Validation MAPE: {valid_mape}, Validation R2: {valid_r2}')

In [ ]:
# results['train']['mse'] = [train_mse]
# results['train']['mape'] = [train_mape]
# results['train']['r2'] = [train_r2]

# results['valid']['mse'] = [valid_mse]
# results['valid']['mape'] = [valid_mape]
# results['valid']['r2'] = [valid_r2]

# results

In [ ]:
# # Plot the training loss
# plt.plot(history.history['loss'])
# plt.show()

In [ ]:
# # Plot the validation loss
# plt.plot(history.history['val_loss'])
# plt.show()

In [ ]:
# # Plot the training rmse loss
# plt.plot(history.history['rmse'])
# plt.show()

In [ ]:
# # Plot the validation rmse loss
# plt.plot(history.history['val_rmse'])
# plt.show()

In [ ]:
# # Plot the training R2 Score
# plt.plot(history.history['r2_score'])
# plt.show()

In [ ]:
# # Plot the validation R2 Score
# plt.plot(history.history['val_r2_score'])
# plt.show()

In [ ]:
# # Plotting the predictions
# plt.figure(figsize=(12, 6))
# plt.plot(y_valid, label='Actual')
# plt.plot(y_valid_pred, label='Predicted')
# plt.legend()
# plt.show()

In [ ]:
# # Plotting for one station, and direction
# mask_station = X_valid[:,0,11]==1
# mask_dir = X_valid[:,0,52]==1
# mask_lane = X_valid[:,0,55]==1
# mask = mask_station & mask_dir & mask_lane

# plt.figure(figsize=(12, 6))
# plt.plot(y_valid[mask], label='Actual')
# plt.plot(y_valid_pred[mask], label='Predicted')
# plt.legend()
# plt.show()

In [ ]:
# # Save the model
# convlstm.save(f'convlstm_{num_convlstm+1}.h5')

# # Save the weights
# convlstm.save_weights('ckpt.weights.h5')